In [1]:
#Imports
import pathlib
import random

import pandas as pd
import tensorflow as tf
import keras_nlp
import keras
#import keras
#from keras import ops

#import tensorflow.data as tf_data
#from tensorflow_text.tools.wordpiece_vocab import (
    #bert_vocab_from_dataset as bert_vocab,
#)

C:\Users\15184\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import string
import tensorflow.strings as tf_strings
import tensorflow.data as tf_data
from keras.layers import TextVectorization
import re
from keras import layers

In [3]:
from keras import ops

In [4]:
#hyper parameters
BATCH_SIZE = 64
EPOCHS = 1  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 40
VOCAB_SIZE = 50000

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

In [5]:
with open('train.tok.fr.txt', encoding="utf8" ) as f:
    fr_lines = f.read().split("\n")[:-1]
f.close()
with open('train.tok.fr.txt', encoding="utf8" ) as f:
    en_lines = f.read().split("\n")[:-1]
f.close()

In [6]:
print(fr_lines[:5])

['', 'I N . U s e r . a u t h o r i z e # f u n c t i o n # # #', 'I N . U s e r . l o g o u t # f u n c t i o n # # #', 'u s e r . e m a i l ▁ # r e s . e m a i l A d d r e s s #', 'u s e r . f i r s t N a m e ▁ # ▁ r e s . l a s t N a m e #']


In [7]:
print(en_lines[:5])

['', 'I N . U s e r . a u t h o r i z e # f u n c t i o n # # #', 'I N . U s e r . l o g o u t # f u n c t i o n # # #', 'u s e r . e m a i l ▁ # r e s . e m a i l A d d r e s s #', 'u s e r . f i r s t N a m e ▁ # ▁ r e s . l a s t N a m e #']


In [8]:
diction = {'fr': fr_lines, 'en':en_lines}
train = pd.DataFrame(diction)

In [9]:
print(train.head())

                                                  fr  \
0                                                      
1  I N . U s e r . a u t h o r i z e # f u n c t ...   
2  I N . U s e r . l o g o u t # f u n c t i o n ...   
3  u s e r . e m a i l ▁ # r e s . e m a i l A d ...   
4  u s e r . f i r s t N a m e ▁ # ▁ r e s . l a ...   

                                                  en  
0                                                     
1  I N . U s e r . a u t h o r i z e # f u n c t ...  
2  I N . U s e r . l o g o u t # f u n c t i o n ...  
3  u s e r . e m a i l ▁ # r e s . e m a i l A d ...  
4  u s e r . f i r s t N a m e ▁ # ▁ r e s . l a ...  


In [10]:
text_pairs = []
for line in train.index:
    text_pairs.append((train['fr'][line],train['en'][line]))

In [11]:
for i in range(2):
    print(random.choice(text_pairs))

('# ▁ a b l e ▁ t o ▁ r e t u r n ▁ a ▁ v a l u e ▁ t h a t ▁ h a s ▁ n o t ▁ b e e n ▁ c o n f i g u r e d ▁ h a s ▁ p r o p e r t y ▁ # e x # ▁ t h e ▁ D O M ▁ n o d e ▁ w h i c h ▁ i d', '# ▁ a b l e ▁ t o ▁ r e t u r n ▁ a ▁ v a l u e ▁ t h a t ▁ h a s ▁ n o t ▁ b e e n ▁ c o n f i g u r e d ▁ h a s ▁ p r o p e r t y ▁ # e x # ▁ t h e ▁ D O M ▁ n o d e ▁ w h i c h ▁ i d')
('S h o w s ▁ a g g r e g a t e ▁ d e t a i l s ▁ o f ▁ e a c h ▁ e a c h ▁ s e t ▁ o f ▁ i t e r a t i o n s #', 'S h o w s ▁ a g g r e g a t e ▁ d e t a i l s ▁ o f ▁ e a c h ▁ e a c h ▁ s e t ▁ o f ▁ i t e r a t i o n s #')


In [12]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) -  num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : ]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")


242306 total pairs
205961 training pairs
36345 validation pairs


In [13]:
with open('dev.tok.fr.txt', encoding="utf8" ) as f:
    fr_lines = f.read().split("\n")[:-1]
f.close()
with open('dev.tok.en.txt', encoding="utf8" ) as f:
    en_lines = f.read().split("\n")[:-1]
f.close()
diction = {'fr': fr_lines, 'en':en_lines}
test = pd.DataFrame(diction)
test_pairs = []
for line in test.index:
    test_pairs.append((test['fr'][line],test['en'][line]))
    
random.shuffle(test_pairs)
print(f"{len(test_pairs)} total pairs")

12752 total pairs


In [14]:
strip_chars = string.punctuation
strip_chars = strip_chars.replace("#", "")

def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")

fr_vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_SEQUENCE_LENGTH,
    standardize=custom_standardization
)

en_vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_SEQUENCE_LENGTH,
    standardize=custom_standardization
)

train_fr_texts = [pair[0] for pair in train_pairs]
train_en_texts = [pair[1] for pair in train_pairs]
fr_vectorization.adapt(train_fr_texts)
en_vectorization.adapt(train_en_texts)

In [15]:
def format_dataset(fr, en):
    fr = fr_vectorization(fr)
    en = en_vectorization(en)
    return (
        {
            "encoder_inputs": fr,
            "decoder_inputs": en,
        },
        en,
    )


def make_dataset(pairs):
    fr_texts, en_texts = zip(*pairs)
    fr_texts = list(fr_texts)
    en_texts = list(en_texts)
    dataset = tf_data.Dataset.from_tensor_slices((fr_texts, en_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
test_ds = make_dataset(test_pairs)

In [16]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 40)
inputs["decoder_inputs"].shape: (64, 40)
targets.shape: (64, 40)


In [17]:
import keras.ops as ops


class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = ops.shape(inputs)[-1]
        positions = ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        if mask is None:
            return None
        else:
            return ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
            padding_mask = ops.minimum(padding_mask, causal_mask)
        else:
            padding_mask = None

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = ops.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = ops.arange(sequence_length)[:, None]
        j = ops.arange(sequence_length)
        mask = ops.cast(i >= j, dtype="int32")
        mask = ops.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = ops.concatenate(
            [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])],
            axis=0,
        )
        return ops.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

In [18]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(MAX_SEQUENCE_LENGTH, VOCAB_SIZE, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(MAX_SEQUENCE_LENGTH, VOCAB_SIZE, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [19]:
epochs = 1 

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs (InputLayer)   │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ positional_embedding          │ (None, None, 256)         │      12,810,240 │ encoder_inputs[0][0]       │
│ (PositionalEmbedding)         │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ decoder_inputs (InputLayer)   │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ transformer_encoder           │ (None, None, 256)         │       3,155,456 │ positional_embedding[0][0] │
│ (TransformerEncoder)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ functional_5 (Functional)     │ (None, None, 50000)       │      30,919,760 │ decoder_inputs[0][0],      │
│                               │                           │                 │ transformer_encoder[0][0]  │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 46,885,456 (178.85 MB)

 Trainable params: 46,885,456 (178.85 MB)

 Non-trainable params: 0 (0.00 B)

3219/3219 ━━━━━━━━━━━━━━━━━━━━ 9541s 3s/step - accuracy: 0.9944 - loss: 0.3382 - val_accuracy: 1.0000 - val_loss: 4.0021e-06


In [20]:
transformer.save_weights("transformermodel.weights.h5")

In [ ]:
#Test accuracy on test data

score, acc = transformer.evaluate(test_ds, batch_size=BATCH_SIZE)
print('Test accuracy with transformers:', acc)

 15/200 ━━━━━━━━━━━━━━━━━━━━ 2:28 802ms/step - accuracy: 1.0000 - loss: 4.1337e-06